## 填充和步幅
一般来说，假设输入的形状是$ n_h \times n_w $，卷积核窗口形状是$ k_h\times k_w $，那么输出的形状是：
<center>
    $ (n_h-k_h+1)\times(n_w-k_w+1) $
</center>

所以卷积层的输出形状由输入形状和卷积核窗口形状决定。本节我们将介绍卷积层的两个超参数，即**填充和步幅**。它们可以对给定形状的输入和卷积核改变输出形状。

### 填充
填充$（padding）$是指在输入高和宽的两侧填充元素（通常是$0$元素）。我们在$5.2$的原输入高和宽的两侧分别添加了值为$0$的元素，使得输入高和宽从$3$变成了$5$，并导致输出高和宽由$2$增加到$4$。$5.2$中的阴影部分为第一个输出元素及其计算所使用的输入和核数组元素：$0\times 0+0\times 1+0\times 2+0\times 3=0$。


<center>
    $ \begin{pmatrix} \mathbf{0}&\mathbf{0}&0&0&0\\\mathbf{0}&\mathbf{0}&1&2&0\\0&3&4&5&0\\0&6&7&8&0\\0&0&0&0&0 \end{pmatrix}$ * $ \begin{pmatrix} 0&1\\2&3 \end{pmatrix}  $ = $ \begin{pmatrix} \mathbf{0}&3&8&4\\9&19&25&10\\21&37&43&16\\6&7&8&0 \end{pmatrix}  $
</center>

一般来说，如果在高的两侧一共填充$p_h$行，在宽的两侧一共填充$p_w$列，那么输出形状将会是

<center>
    $ (n_h+p_h-k_h+1)\times (n_w+p+w-k_w+1) $
</center>

也就是说，输出的高和宽会分别增加$p_h$和$p_w$

在很多情况下，我们会设置$p_h=k_h−1$和$p_w=k_w−1$来使输入和输出具有相同的高和宽。这样会方便在构造网络时推测每个层的输出形状。假设这里$k_h$是奇数，我们会在高的两侧分别填充$p_h/2$行。如果$k_h$是偶数，一种可能是在输入的顶端一侧填充$⌈p_h/2⌉$行，而在底端一侧填充$⌊p_h/2⌋$行。在宽的两侧填充同理。

卷积神经网络经常使用奇数高宽的卷积核，如$1$、$3$、$5$和$7$，所以两端上的填充个数相等。对任意的二维数组$X$，设它的第$i$行第$j$列的元素为$X[i,j]$。**当两端上的填充个数相等，并使输入和输出具有相同的高和宽时，我们就知道输出$Y[i,j]$是由输入以$X[i,j]$为中心的窗口同卷积核进行互相关计算得到的**。

下面的例子里我们创建一个高和宽为$3$的二维卷积层，然后设输入高和宽两侧的填充数分别为$1$。给定一个高和宽为$8$的输入，我们发现输出的高和宽也是$8$。

In [1]:
import torch
from torch import nn

def comp_conv2d(conv2d, X):
     # (1, 1)代表批量大小和通道数（“多输入通道和多输出通道”一节将介绍）均为1
    X = X.view((1,1)+X.shape)
    Y = conv2d(X)
    return Y.view(Y.shape[2:])# 排除不关心的前两维：批量和通道

# 注意这里是两侧分别填充1行或列，所以在两侧一共填充2行或列
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)
X = torch.rand(8, 8)
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

当卷积核的高和宽不同时，我们也可以通过设置高和宽上不同的填充数使输出和输入具有相同的高和宽。

In [2]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5,3), padding=(2,1))
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])

### 步幅
在上一节里我们介绍了二维互相关运算。卷积窗口从输入数组的最左上方开始，按从左往右、从上往下的顺序，依次在输入数组上滑动。我们将每次滑动的行数和列数称为步幅$（stride）$。

一般来说，当高上步幅为$s_h$，宽上步幅为$s_w$时，输出形状为
<center>
   $\lfloor (n_h+p_h-k_h+s_h)/s_h \rfloor\times \lfloor (n_w+p_w-k_w+s_w)/s_w \rfloor$
</center>

下面我们令宽和高上的步幅均为$2$，从而使输入的高和宽减半。

In [3]:
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [4]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])

为了表述简洁，当输入的高和宽两侧的填充数分别为$p_h$和$p_w$时，我们称填充为$（p_h, p_w）$。特别地，当$p_h=p_w=p$时，填充为$p$。当在高和宽上的步幅分别为$s_h$和$s_w$时，我们称步幅为$（s_h, s_w）$。特别地，当$s_h=s_w=s$时，步幅为$s$。在默认情况下，填充为$0$，步幅为$1$。

+ 填充可以增加输出的高和宽。这常用来使输出与输入具有相同的高和宽。
+ 步幅可以减小输出的高和宽，例如输出的高和宽仅为输入的高和宽的$1/n$（$n$为大于$1$的整数）。